In [3]:
println("cLASIFICACIÓN EJERCICIOS")
#=
LASSO
Ridge
Elastic Net
Decision Tree
Random Forest
Nearest Neighbors
Support Vector Machines (SVM)

You will need to do the following

Get the data
Clean it
Use Go or Julia to write/use the classification algos. 
Evaluate the accuracy of each prediction, using a confusion matrix and a ROC curve.
Submit your project following the submission guidelines
=#

# Importar librerias 
import Pkg; using Pkg
#Pkg.add("DelimitedFiles"); using DelimitedFiles  //less powerfuul library than csv 
Pkg.add("DataFrames"); using DataFrames
Pkg.add("CSV"); using CSV
Pkg.add("Clustering"); using Clustering
Pkg.add("Statistics"); using Statistics
Pkg.add("JSON"); using JSON
Pkg.add("Distances"); using Distances
Pkg.add("GLMNet"); using GLMNet
Pkg.add("MLBase"); using MLBase
Pkg.add("Random"); using Random
Pkg.offline(true)


cLASIFICACIÓN EJERCICIOS


   Resolving package versions...
  No Changes to `~/Desktop/Ejercicios/Exercise3/Project.toml`
  No Changes to `~/Desktop/Ejercicios/Exercise3/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Desktop/Ejercicios/Exercise3/Project.toml`
  No Changes to `~/Desktop/Ejercicios/Exercise3/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Desktop/Ejercicios/Exercise3/Project.toml`
  No Changes to `~/Desktop/Ejercicios/Exercise3/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Desktop/Ejercicios/Exercise3/Project.toml`
  No Changes to `~/Desktop/Ejercicios/Exercise3/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Desktop/Ejercicios/Exercise3/Project.toml`
  No Changes to `~/Desktop/Ejercicios/Exercise3/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Desktop/Ejercicios/Exercise3/Project.toml`
  No Changes to `~/Desktop/Ejercicios/Exercise3/Manifest.toml`
   Resolving package versions...
  No Changes to `~/

In [125]:
#assign_class(predictidValues) = argmin(abs.(predictidValues .-[0,1]))

function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for i in uids
        curids = findall(y.==i)
        row_ids = randsubseq(curids, at)
        push!(keepids, getindex.(row_ids, 1)...)
    end 

    return keepids  
end

function findaccuracy(predictions, actual)
    correct = sum(predictions .== actual)
    accuracy = correct / length(actual)
    return accuracy
end

function confusion_matrix(actual, predicted)
    # Obtener las etiquetas únicas
    labels = unique(vcat(actual, predicted))
    n_classes = length(labels)
    
    # Inicializar matriz de confusión
    conf_matrix = zeros(Int, n_classes, n_classes)
    
    # Llenar la matriz de confusión
    for (act, pred) in zip(actual, predicted)
        i = findfirst(==(act), labels)
        j = findfirst(==(pred), labels)
        conf_matrix[i,j] += 1
    end
    
    # Agregar encabezados
    println("\nMatriz de Confusión:")
    println("Predicho →")
    println("Real ↓")
    println("     ", join([string(i) for i in labels], "   "))
    for i in 1:n_classes
        print(labels[i], "  ")
        println(join([string(conf_matrix[i,j]) for j in 1:n_classes], "   "))
    end
    
    return conf_matrix
end


function graficarCurvaROC(gt,pred,title)
    # Calcular la curva ROC
roc_curve = MLBase.roc(gt, pred)

# Extraer los puntos de la curva ROC
fpr = [r.fp / (r.fp + r.tn) for r in roc_curve]  # Tasa de Falsos Positivos
tpr = [r.tp / (r.tp + r.fn) for r in roc_curve]  # Tasa de Verdaderos Positivos

# Graficar la curva ROC
plot(fpr, tpr,
     label="Curva ROC",
     xlabel="Tasa de Falsos Positivos (1-Especificidad)",
     ylabel="Tasa de Verdaderos Positivos (Sensibilidad)", 
     title="Curva ROC",
     legend=:bottomright)

# Agregar línea diagonal de referencia
plot!([0, 1], [0, 1],
      label="Línea Base",
      linestyle=:dash,
      color=:gray)

savefig("../fig/ROC_$title.png")
end


graficarCurvaROC (generic function with 1 method)

In [4]:
market = CSV.read("../dat/Smarket.csv", DataFrame);
size(market)

(1250, 9)

In [6]:
#println(names(C))
@show println(describe(market))
a = size(market)
@show names(market)

9×7 DataFrame
 Row │ variable   mean       min      median   max      nmissing  eltype   
     │ Symbol     Union…     Any      Union…   Any      Int64     DataType 
─────┼─────────────────────────────────────────────────────────────────────
   1 │ Year       2003.02    2001     2003.0   2005            0  Int64
   2 │ Lag1       0.0038344  -4.922   0.039    5.733           0  Float64
   3 │ Lag2       0.0039192  -4.922   0.039    5.733           0  Float64
   4 │ Lag3       0.001716   -4.922   0.0385   5.733           0  Float64
   5 │ Lag4       0.001636   -4.922   0.0385   5.733           0  Float64
   6 │ Lag5       0.0056096  -4.922   0.0385   5.733           0  Float64
   7 │ Volume     1.4783     0.35607  1.42295  3.15247         0  Float64
   8 │ Today      0.0031384  -4.922   0.0385   5.733           0  Float64
   9 │ Direction             Down              Up              0  String7
println(describe(market)) = nothing
names(market) = ["Year", "Lag1", "Lag2", "Lag3", "Lag4", "

9-element Vector{String}:
 "Year"
 "Lag1"
 "Lag2"
 "Lag3"
 "Lag4"
 "Lag5"
 "Volume"
 "Today"
 "Direction"

In [7]:
x = Matrix(market[!, [:Lag1, :Lag2, :Lag3, :Lag4, :Lag5, :Volume, :Today]])
market_labels = Matrix(market[!, [:Direction]])


1250×1 Matrix{String7}:
 "Up"
 "Up"
 "Down"
 "Up"
 "Up"
 "Up"
 "Down"
 "Up"
 "Up"
 "Up"
 ⋮
 "Down"
 "Down"
 "Up"
 "Up"
 "Up"
 "Down"
 "Up"
 "Down"
 "Down"

In [128]:
#Valores unicos de la columna Direction
marketLabelsMap = labelmap(market_labels)
y = map(v -> v == "Down" ? 0 : 1, market_labels)



1250×1 Matrix{Int64}:
 1
 1
 0
 1
 1
 1
 0
 1
 1
 1
 ⋮
 0
 0
 1
 1
 1
 0
 1
 0
 0

In [129]:
x

1250×7 Matrix{Float64}:
  0.381  -0.192  -2.624  -1.055   5.01   1.1913    0.959
  0.959   0.381  -0.192  -2.624  -1.055  1.2965    1.032
  1.032   0.959   0.381  -0.192  -2.624  1.4112   -0.623
 -0.623   1.032   0.959   0.381  -0.192  1.276     0.614
  0.614  -0.623   1.032   0.959   0.381  1.2057    0.213
  0.213   0.614  -0.623   1.032   0.959  1.3491    1.392
  1.392   0.213   0.614  -0.623   1.032  1.445    -0.403
 -0.403   1.392   0.213   0.614  -0.623  1.4078    0.027
  0.027  -0.403   1.392   0.213   0.614  1.164     1.303
  1.303   0.027  -0.403   1.392   0.213  1.2326    0.287
  ⋮                                      ⋮        
 -0.285  -0.141   0.419   0.555   0.084  2.58419  -0.584
 -0.584  -0.285  -0.141   0.419   0.555  2.20881  -0.024
 -0.024  -0.584  -0.285  -0.141   0.419  1.99669   0.252
  0.252  -0.024  -0.584  -0.285  -0.141  2.06517   0.422
  0.422   0.252  -0.024  -0.584  -0.285  1.8885    0.043
  0.043   0.422   0.252  -0.024  -0.584  1.28581  -0.955
 -0.955   0.0

In [130]:
train_ids = perclass_splits(y, 0.7)
test_ids = setdiff(1:length(y,), train_ids)

train_x = x[train_ids,:]
train_y = y[train_ids]
test_x = x[test_ids,:]
test_y = y[test_ids]



362-element Vector{Int64}:
 1
 0
 0
 0
 1
 0
 0
 0
 1
 1
 ⋮
 0
 1
 1
 1
 0
 1
 1
 0
 0

In [81]:
#probando el modelo LASSO
path = glmnet(train_x, train_y )
cv = glmnetcv(train_x, train_y)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(train_x, train_y, lambda=[mylambda])


Least Squares GLMNet Solution Path (1 solutions for 7 predictors in 4 passes):
────────────────────────────
     df   pct_dev          λ
────────────────────────────
[1]   1  0.527549  0.0203305
────────────────────────────

In [133]:
using MLBase

q = x[test_ids,:]
predictions_lasso_prob   =GLMNet.predict(path, q)
predictions_lasso = map(v -> v > .5 ? 1 : 0, predictions_lasso_prob)

@show findaccuracy(predictions_lasso, test_y)
gt = vec(test_y) 
pred = vec(predictions_lasso_prob)
graficarCurvaROC(gt,pred,"LASSO")


findaccuracy(predictions_lasso, test_y) = 0.9751381215469613


"/home/cecilia/Desktop/Ejercicios/Exercise3/fig/ROC_LASSO.png"

In [134]:
# Ejemplo de uso
conf_mat = confusion_matrix(test_y, predictions_lasso)



Matriz de Confusión:
Predicho →
Real ↓
     1   0
1  181   1
0  8   172


2×2 Matrix{Int64}:
 181    1
   8  172

In [135]:
#Method 2: Ridge
#We will use the same function but set alpha to zero.

# choose the best lambda to predict with.
path = glmnet(train_x, train_y,alpha=0);
cv = glmnetcv(train_x, train_y,alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(train_x, train_y,alpha=0,lambda=[mylambda]);
q = test_x;
predictions_ridge_prob   = GLMNet.predict(path,q)
predictions_ridge = map(v -> v > .5 ? 1 : 0, predictions_ridge_prob)
findaccuracy(predictions_ridge,test_y)
     


0.9392265193370166

In [136]:
matriz_confusion = confusion_matrix(test_y, predictions_ridge)


Matriz de Confusión:
Predicho →
Real ↓
     1   0
1  181   1
0  21   159


2×2 Matrix{Int64}:
 181    1
  21  159

In [144]:
gt = vec(test_y) 
pred = vec(predictions_ridge_prob)
graficarCurvaROC(gt,pred,"RIDGE")


"/home/cecilia/Desktop/Ejercicios/Exercise3/fig/ROC_RIDGE.png"

In [138]:
#Method 3: Elastic Net
#We will use the same function but set alpha to 0.5 (it's the combination of lasso and ridge).

#choose the best lambda to predict with.
path = glmnet(train_x, train_y,alpha=0.5);
cv = glmnetcv(train_x, train_y,alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(train_x, train_y,alpha=0.5,lambda=[mylambda]);
q = test_x;
predictions_EN_prob = GLMNet.predict(path,q)
predictions_EN = map(v -> v > .5 ? 1 : 0, predictions_EN_prob)
findaccuracy(predictions_EN,test_y)
         
    

0.9502762430939227

In [141]:
matriz_confusion = confusion_matrix(test_y, predictions_EN)




Matriz de Confusión:
Predicho →
Real ↓
     1   0
1  182   0
0  18   162


2×2 Matrix{Int64}:
 182    0
  18  162

In [143]:
gt = vec(test_y) 
pred = vec(predictions_EN_prob)
graficarCurvaROC(gt,pred,"ELASTIC_NET")

"/home/cecilia/Desktop/Ejercicios/Exercise3/fig/ROC_ELASTIC_NET.png"

In [145]:
# Method 4: Decision Tree
# We will use the package DecisionTree
Pkg.add("DecisionTree"); using DecisionTree
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, train_x, train_y)
q = test_x;
predictions_DT_prob = DecisionTree.predict(model, q)
predictions_DT = map(v -> v > .5 ? 1 : 0, predictions_DT_prob)
findaccuracy(predictions_DT,test_y)





   Resolving package versions...
  No Changes to `~/Desktop/Ejercicios/Exercise3/Project.toml`
  No Changes to `~/Desktop/Ejercicios/Exercise3/Manifest.toml`


1.0

In [146]:
matriz_confusion = confusion_matrix(test_y, predictions_DT)



Matriz de Confusión:
Predicho →
Real ↓
     1   0
1  182   0
0  0   180


2×2 Matrix{Int64}:
 182    0
   0  180

In [148]:
gt = vec(test_y) 
pred = vec(predictions_DT_prob)
pred = Float64.(pred)  # se hizo la converción porque lo converia automaticamente a int
graficarCurvaROC(gt,pred,"DECISION_TREE")


"/home/cecilia/Desktop/Ejercicios/Exercise3/fig/ROC_DECISION_TREE.png"

In [149]:
#Method 5: Random Forests
#The RandomForestClassifier is available through the DecisionTree package as well.

model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, train_x, train_y)

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [0, 1]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 7.5
Avg Depth:  2.9

In [150]:
q = test_x;
predictions_RF_prob = DecisionTree.predict(model, q)
predictions_RF = map(v -> v > .5 ? 1 : 0, predictions_RF_prob)
findaccuracy(predictions_RF,test_y)


1.0

In [156]:
matriz_confusion = confusion_matrix(test_y, predictions_RF)

gt = vec(test_y) 
pred = vec(predictions_RF_prob)
pred = Float64.(pred)  # se hizo la converción porque lo converia automaticamente a int
graficarCurvaROC(gt,pred,"RANDOM_FOREST")


Matriz de Confusión:
Predicho →
Real ↓
     1   0
1  182   0
0  0   180


"/home/cecilia/Desktop/Ejercicios/Exercise3/fig/ROC_RANDOM_FOREST.png"

In [151]:
#Method 6: Using a Nearest Neighbor method
#We will use the NearestNeighbors package here.
Pkg.add("NearestNeighbors"); using NearestNeighbors

kdtree = KDTree(train_x')

   Resolving package versions...
  No Changes to `~/Desktop/Ejercicios/Exercise3/Project.toml`
  No Changes to `~/Desktop/Ejercicios/Exercise3/Manifest.toml`


KDTree{StaticArraysCore.SVector{7, Float64}, Euclidean, Float64, StaticArraysCore.SVector{7, Float64}}
  Number of points: 888
  Dimensions: 7
  Metric: Euclidean(0.0)
  Reordered: true

In [152]:
queries = test_x


362×7 Matrix{Float64}:
  0.381  -0.192  -2.624  -1.055   5.01   1.1913    0.959
  0.287   1.303   0.027  -0.403   1.392  1.309    -0.498
 -0.498   0.287   1.303   0.027  -0.403  1.258    -0.189
 -0.151   0.359  -1.747   0.546  -0.562  1.0596   -0.841
 -1.334  -0.623  -0.841  -0.151   0.359  1.0755    1.183
 -1.736  -1.891   0.812  -0.218  -0.865  1.1122   -1.851
 -0.766   1.749  -0.556  -0.195  -1.851  1.1141   -1.431
  0.104  -1.431  -0.766   1.749  -0.556  1.2949   -0.568
  0.998   0.586  -0.568   0.104  -1.431  1.0918    0.645
  0.645   0.998   0.586  -0.568   0.104  1.1322    0.226
  ⋮                                      ⋮        
  0.002  -0.851   0.209   0.347   0.508  2.26834  -0.636
  1.216  -0.636   0.002  -0.851   0.209  2.61483   0.032
 -0.236   0.032   1.216  -0.636   0.002  2.32584   0.128
  0.281  -0.122  -0.501   0.128  -0.236  1.89629   0.084
  0.419   0.555   0.084   0.281  -0.122  2.14552  -0.141
 -0.024  -0.584  -0.285  -0.141   0.419  1.99669   0.252
  0.422   0.25

In [153]:
idxs, dists = knn(kdtree, queries', 5, true)

([[161, 590, 120, 15, 23], [545, 733, 827, 695, 749], [727, 246, 736, 813, 809], [572, 796, 557, 773, 705], [214, 303, 388, 301, 183], [839, 664, 506, 595, 583], [523, 644, 830, 789, 762], [741, 501, 645, 747, 799], [36, 245, 80, 323, 227], [380, 368, 759, 823, 263]  …  [452, 440, 377, 427, 421], [886, 819, 863, 887, 782], [435, 835, 399, 861, 443], [391, 438, 408, 867, 422], [465, 410, 425, 453, 445], [851, 870, 443, 410, 852], [462, 427, 783, 452, 440], [375, 280, 394, 463, 465], [863, 769, 682, 782, 317], [850, 793, 802, 244, 304]], [[3.312866046929758, 3.494218974248752, 3.600970953784548, 3.644006625954459, 3.79221927636048], [0.8125574687368272, 0.8298058869398311, 0.9865580717322219, 1.0227550097652907, 1.146348393814027], [0.6590459847385461, 0.837463366362971, 0.8424281868503688, 0.8530572137904936, 0.8925271368423484], [0.9217248450595221, 1.004630598777481, 1.1165131839794817, 1.1806212093639519, 1.2036497040252203], [1.2161660289615066, 1.2696346561117493, 1.289008325186459

In [175]:
c = train_y[hcat(idxs...)]
#possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Float32,string(string(argmax(c[i])))),1:size(c,2))
findaccuracy(predictions_NN,test_y)

0.5027624309392266

In [178]:

matriz_confusion = confusion_matrix(test_y, predictions_NN)



Matriz de Confusión:
Predicho →
Real ↓
     1.0   0.0
1.0  182   0
0.0  180   0


2×2 Matrix{Int64}:
 182  0
 180  0

In [159]:
#Method 7: Support Vector Machines
#We will use the LIBSVM package here.
Pkg.add("LIBSVM"); using LIBSVM

model = svmtrain(train_x', train_y)



   Resolving package versions...
  No Changes to `~/Desktop/Ejercicios/Exercise3/Project.toml`
  No Changes to `~/Desktop/Ejercicios/Exercise3/Manifest.toml`


LIBSVM.SVM{Int64, LIBSVM.Kernel.KERNEL}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 7, 888, 2, [1, 0], Int32[1, 2], Float64[], Int32[], LIBSVM.SupportVectors{Vector{Int64}, Matrix{Float64}}(305, Int32[151, 154], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-0.403 1.303 … -0.141 -0.584; 1.392 0.027 … 0.419 -0.285; … ; 1.4078 1.2326 … 2.18059 2.20881; 0.027 0.287 … -0.285 -0.024], Int32[5, 7, 14, 17, 18, 19, 21, 22, 27, 30  …  865, 870, 872, 874, 877, 880, 882, 884, 885, 887], LIBSVM.SVMNode[LIBSVM.SVMNode(1, -0.403), LIBSVM.SVMNode(1, 1.303), LIBSVM.SVMNode(1, -1.431), LIBSVM.SVMNode(1, -4.318), LIBSVM.SVMNode(1, -2.584), LIBSVM.SVMNode(1, -1.962), LIBSVM.SVMNode(1, -0.29), LIBSVM.SVMNode(1, -0.213), LIBSVM.SVMNode(1, 1.359), LIBSVM.SVMNode(1, 2.845)  …  LIBSVM.SVMNode(1, 0.796), LIBSVM.SVMNode(1, 0.092), LIBSVM.SVMNode(1, -1.489), LIBSVM.SVMNode(1, -0.717), LIBSVM.SVMNode(1, -0.237), LIBSVM.SVMNode(1, 0.305), LIBSVM.SVMNode(1, 0.032), LIBSVM.SVMNode(1, -0.501), LIBSVM

In [170]:
predictions_SVM, decision_values  = svmpredict(model, test_x')
findaccuracy(predictions_SVM,test_y)

0.9475138121546961

In [171]:

matriz_confusion = confusion_matrix(test_y, predictions_SVM)





Matriz de Confusión:
Predicho →
Real ↓
     1   0
1  178   4
0  15   165


2×2 Matrix{Int64}:
 178    4
  15  165

In [180]:
overall_accuracies = zeros(7)
methods = ["lasso","ridge","EN", "DT", "RF","kNN", "SVM"]
#ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso, test_y)
overall_accuracies[2] = findaccuracy(predictions_ridge,test_y)
overall_accuracies[3] = findaccuracy(predictions_EN,test_y)
overall_accuracies[4] = findaccuracy(predictions_DT,test_y)
overall_accuracies[5] = findaccuracy(predictions_RF,test_y)
overall_accuracies[6] = findaccuracy(predictions_NN,test_y)
overall_accuracies[7] = findaccuracy(predictions_SVM,test_y)
hcat(methods, overall_accuracies)

7×2 Matrix{Any}:
 "lasso"  0.975138
 "ridge"  0.939227
 "EN"     0.950276
 "DT"     1.0
 "RF"     1.0
 "kNN"    0.502762
 "SVM"    0.947514